In [2]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

from transformers import pipeline
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [3]:
df = pd.read_csv("/Users/chandlershortlidge/Desktop/Ironhack/project-dsai-business-case-automated-customer-reviews/data/amazon-customer-reviews/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")

# EDA

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28332 entries, 0 to 28331
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   28332 non-null  object 
 1   dateAdded            28332 non-null  object 
 2   dateUpdated          28332 non-null  object 
 3   name                 28332 non-null  object 
 4   asins                28332 non-null  object 
 5   brand                28332 non-null  object 
 6   categories           28332 non-null  object 
 7   primaryCategories    28332 non-null  object 
 8   imageURLs            28332 non-null  object 
 9   keys                 28332 non-null  object 
 10  manufacturer         28332 non-null  object 
 11  manufacturerNumber   28332 non-null  object 
 12  reviews.date         28332 non-null  object 
 13  reviews.dateSeen     28332 non-null  object 
 14  reviews.didPurchase  9 non-null      object 
 15  reviews.doRecommend  16086 non-null 

In [5]:
df.shape

(28332, 24)

In [6]:
df.columns = df.columns.str.lower()

In [7]:
df.head()

,id,dateadded,dateupdated,name,asins,brand,categories,primarycategories,imageurls,keys,...,reviews.didpurchase,reviews.dorecommend,reviews.id,reviews.numhelpful,reviews.rating,reviews.sourceurls,reviews.text,reviews.title,reviews.username,sourceurls
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [8]:
df.isnull().sum()

id                         0
dateadded                  0
dateupdated                0
name                       0
asins                      0
brand                      0
categories                 0
primarycategories          0
imageurls                  0
keys                       0
manufacturer               0
manufacturernumber         0
reviews.date               0
reviews.dateseen           0
reviews.didpurchase    28323
reviews.dorecommend    12246
reviews.id             28291
reviews.numhelpful     12217
reviews.rating             0
reviews.sourceurls         0
reviews.text               0
reviews.title              0
reviews.username           5
sourceurls                 0
dtype: int64

In [9]:
df = df.drop(columns=["reviews.didpurchase", "reviews.id", "reviews.numhelpful", "reviews.dorecommend"])

In [10]:
df.isnull().sum()

id                    0
dateadded             0
dateupdated           0
name                  0
asins                 0
brand                 0
categories            0
primarycategories     0
imageurls             0
keys                  0
manufacturer          0
manufacturernumber    0
reviews.date          0
reviews.dateseen      0
reviews.rating        0
reviews.sourceurls    0
reviews.text          0
reviews.title         0
reviews.username      5
sourceurls            0
dtype: int64

# Create sentiment categories

In [11]:
df["reviews.rating"].value_counts()

reviews.rating
5    19897
4     5648
3     1206
1      965
2      616
Name: count, dtype: int64

In [12]:
sentiment_map = {1: "negative", 2: "negative", 3: "neutral", 4: "positive", 5: "positive"}
df["rating_sentiment"] = df["reviews.rating"].map(sentiment_map)

In [13]:
df.head()

,id,dateadded,dateupdated,name,asins,brand,categories,primarycategories,imageurls,keys,...,manufacturernumber,reviews.date,reviews.dateseen,reviews.rating,reviews.sourceurls,reviews.text,reviews.title,reviews.username,sourceurls,rating_sentiment
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,HL-002619,2017-03-02T00:00:00.000Z,2017-08-28T00:00:00Z,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht...",neutral
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,HL-002619,2016-08-31T00:00:00.000Z,2017-08-28T00:00:00Z,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht...",positive
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,HL-002619,2016-12-06T00:00:00.000Z,2017-08-28T00:00:00Z,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht...",positive
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,HL-002619,2016-03-28T00:00:00.000Z,2017-08-28T00:00:00Z,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht...",positive
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,HL-002619,2017-04-22T00:00:00.000Z,2017-08-28T00:00:00Z,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht...",positive


In [14]:
df["rating_sentiment"].value_counts()

rating_sentiment
positive    25545
negative     1581
neutral      1206
Name: count, dtype: int64

In [15]:
percentage_of = round((df["rating_sentiment"].value_counts() / len(df) * 100), 2)

percentage_of

# very imbalanced dataset

rating_sentiment
positive    90.16
negative     5.58
neutral      4.26
Name: count, dtype: float64

# Downsample data for balanced classes

In [16]:
# Split the dataframe into three: positive, negative, neutral
positive = (df[df["rating_sentiment"] == "positive"])
negative = (df[df["rating_sentiment"] == "negative"])
neutral =  (df[df["rating_sentiment"] == "neutral"])

positive_sampled = positive.sample(n=1206)
negative_sampled = negative.sample(n=1206)

len(positive_sampled)

# concat into one balanced dataframe 

df_balanced = pd.concat([negative_sampled, positive_sampled, neutral])

# confirm
df_balanced["rating_sentiment"].value_counts()

rating_sentiment
negative    1206
positive    1206
neutral     1206
Name: count, dtype: int64

# Choose a classifier model

- 0: Negative
- 1: Neutral
- 2: Positive

In [17]:
# pick a model 
# CardiffNLP baseline tokenizes the model
classifier = pipeline(task="sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")

# view a sample 

sample_reviews = df_balanced["reviews.text"].head(10).tolist()

results = classifier(sample_reviews)
results


Device set to use mps:0


[{'label': 'LABEL_0', 'score': 0.9769663214683533},
 {'label': 'LABEL_0', 'score': 0.9356447458267212},
 {'label': 'LABEL_0', 'score': 0.9546186327934265},
 {'label': 'LABEL_0', 'score': 0.8882050514221191},
 {'label': 'LABEL_0', 'score': 0.9266035556793213},
 {'label': 'LABEL_1', 'score': 0.6839084029197693},
 {'label': 'LABEL_0', 'score': 0.5865965485572815},
 {'label': 'LABEL_0', 'score': 0.5851985216140747},
 {'label': 'LABEL_0', 'score': 0.8517378568649292},
 {'label': 'LABEL_1', 'score': 0.627692699432373}]

In [18]:
sample_sentiment = df_balanced["rating_sentiment"].head(10).tolist
sample_sentiment

<bound method IndexOpsMixin.tolist of 8746     negative
7959     negative
990      negative
25787    negative
7203     negative
23776    negative
6784     negative
18751    negative
13845    negative
28254    negative
Name: rating_sentiment, dtype: object>

In [19]:
full_results = classifier(df_balanced["reviews.text"].tolist(), truncation=True, max_length=512)

In [20]:
label_strings = [r["label"] for r in full_results]
# returns a list of strings
label_strings
# turn the list of strings into a Series using pd.Series
labels_for_mapping = pd.Series(label_strings)
# now labels are ready for mapping
labels_for_mapping


0       LABEL_0
1       LABEL_0
2       LABEL_0
3       LABEL_0
4       LABEL_0
         ...   
3613    LABEL_0
3614    LABEL_0
3615    LABEL_0
3616    LABEL_0
3617    LABEL_2
Length: 3618, dtype: object

In [21]:
# map the labels into sentiment and create a new column based on the predicted sentiment
label_map = {"LABEL_0": "negative", "LABEL_1": "neutral", "LABEL_2": "positive"}
df_balanced["predicted_sentiment"] = labels_for_mapping.map(label_map).values

In [22]:
predicted = df_balanced["predicted_sentiment"]
actual = df_balanced["rating_sentiment"]
print(classification_report(actual, predicted))

              precision    recall  f1-score   support

    negative       0.68      0.75      0.71      1206
     neutral       0.52      0.26      0.35      1206
    positive       0.65      0.91      0.76      1206

    accuracy                           0.64      3618
   macro avg       0.62      0.64      0.61      3618
weighted avg       0.62      0.64      0.61      3618



### Export class-balanced dataset to csv for use in Colab

In [23]:
df_balanced.to_csv('balanced_reviews.csv', index=False)

# Fine tuning

# 2. Product Category Clustering

In [27]:
df["categories"]

0        AA,AAA,Health,Electronics,Health & Household,C...
1        AA,AAA,Health,Electronics,Health & Household,C...
2        AA,AAA,Health,Electronics,Health & Household,C...
3        AA,AAA,Health,Electronics,Health & Household,C...
4        AA,AAA,Health,Electronics,Health & Household,C...
                               ...                        
28327    Fire Tablets,Tablets,All Tablets,Amazon Tablet...
28328    Fire Tablets,Tablets,All Tablets,Amazon Tablet...
28329    Fire Tablets,Tablets,All Tablets,Amazon Tablet...
28330    Fire Tablets,Tablets,All Tablets,Amazon Tablet...
28331    Fire Tablets,Tablets,All Tablets,Amazon Tablet...
Name: categories, Length: 28332, dtype: object

In [25]:
df["categories"].value_counts()

categories
AA,AAA,Health,Electronics,Health & Household,Camcorder Batteries,Camera & Photo,Batteries,Household Batteries,Robot Check,Accessories,Camera Batteries,Health and Beauty,Household Supplies,Batteries & Chargers,Health, Household & Baby Care,Health Personal Care                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           8343
AA,AAA,Electronics Features,Health,Electro

### Preprocess the text for embedding and clustering

In [ ]:
# split and explode categores to produce a series of words
categories = df["categories"].str.split(",").explode()
categories

0                                AA
0                               AAA
0                            Health
0                       Electronics
0                Health & Household
                    ...            
28331                Amazon Tablets
28331    Genuine Tablet Accessories
28331           Computers & Tablets
28331                   Electronics
28331                        Amazon
Name: categories, Length: 401756, dtype: object

In [ ]:
# remove the top 10 words from the series 
# the words overpower each cluster, causing redundencies
top_10words = categories.value_counts().head(10)
remove_terms = top_10words.index.tolist()
remove_terms


['Electronics',
 'Health',
 'Computers & Tablets',
 'Tablets',
 'Fire Tablets',
 'Amazon Tablets',
 'Accessories',
 'AA',
 'Household Supplies',
 'AAA']

In [ ]:
# goal: replace the categories with a new category list minus removed words

df["categories_clean"] = df["categories"] # copy category list to new column

for term in remove_terms: # for each word in the list 
    df["categories_clean"] = df["categories_clean"].str.replace(term, "", regex=False) # replace term with blank

# now we can vectorize the clean categories column

## Vectorize the text to prep for modeling

In [117]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

text = df["categories_clean"].str.strip().str.lower()

embeddings_cleaned = model.encode(text)

In [109]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4, random_state=42)

kmeans.fit(embeddings_cleaned)

labels = kmeans.labels_

df["cluster_cleaned"] = labels

df["cluster_cleaned"].value_counts()

cluster_cleaned
1    13492
2     8345
0     3728
3     2767
Name: count, dtype: int64

In [120]:
for i in range(kmeans.n_clusters):
    print(f"\n=== Cluster {i} ===")
    print(df[df["cluster_cleaned"] == i]["categories"].head(10).tolist())



=== Cluster 0 ===
['AA,AAA,Electronics Features,Health,Electronics,Health & Household,Camcorder Batteries,Camera & Photo,Batteries,Household Batteries,Accessories,Camera Batteries,Health and Beauty,Household Supplies,Batteries & Chargers,Health, Household & Baby Care,Health Personal Care', 'AA,AAA,Electronics Features,Health,Electronics,Health & Household,Camcorder Batteries,Camera & Photo,Batteries,Household Batteries,Accessories,Camera Batteries,Health and Beauty,Household Supplies,Batteries & Chargers,Health, Household & Baby Care,Health Personal Care', 'AA,AAA,Electronics Features,Health,Electronics,Health & Household,Camcorder Batteries,Camera & Photo,Batteries,Household Batteries,Accessories,Camera Batteries,Health and Beauty,Household Supplies,Batteries & Chargers,Health, Household & Baby Care,Health Personal Care', 'AA,AAA,Electronics Features,Health,Electronics,Health & Household,Camcorder Batteries,Camera & Photo,Batteries,Household Batteries,Accessories,Camera Batteries,Hea

In [ ]:
batteries_and_cameras = df[df["cluster_cleaned"] == 0]["categories_clean"].str.split(",").explode().str.strip().value_counts().head(10)

categories_clean
                       22368
A                       3728
Features                3728
& Household             3728
Camcorder Batteries     3728
Camera & Photo          3728
Batteries               3728
Household Batteries     3728
Camera Batteries        3728
and Beauty              3728
Name: count, dtype: int64

In [ ]:
electronics = df[df["cluster_cleaned"] == 1]["categories_clean"].str.split(",").explode().str.strip().value_counts().head(10)


categories_clean
                           40018
Amazon                     14745
Fire                       11629
All                         9104
iPad &                      7107
Features                    5440
Computers/ & Networking     5195
& eBook Readers             5003
Music                       3702
Tech Toys                   3702
Name: count, dtype: int64

In [ ]:
household_beauty = df[df["cluster_cleaned"] == 2]["categories_clean"].str.split(",").explode().str.strip().value_counts().head(10)
print(f"---Cluster 2: Household & Personal---")
print()
print(cluster_2)

---Cluster 2: Household & Personal---

categories_clean
                         50058
Robot Check               8343
Personal Care             8343
Household & Baby Care     8343
Batteries & Chargers      8343
A                         8343
Camera Batteries          8343
and Beauty                8343
Household Batteries       8343
Batteries                 8343
Name: count, dtype: int64


In [ ]:
kids = df[df["cluster_cleaned"] == 3]["categories_clean"].str.split(",").explode().str.strip().value_counts().head(10)

categories_clean
          5467
Fire      2767
Amazon    2767
All       2474
Kids'      293
Name: count, dtype: int64

# Non-preprocessed k-means

In [131]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

categories = df["categories"].str.strip().str.lower()

embeddings = model.encode(categories)

In [132]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4, random_state=42)

kmeans.fit(embeddings)

labels = kmeans.labels_

df["clusters"] = labels

df["clusters"].value_counts()

clusters
1    11196
2     8343
3     5065
0     3728
Name: count, dtype: int64

In [133]:
for i in range(kmeans.n_clusters):
    print(f"\n=== Cluster {i} ===")
    print(df[df["clusters"] == i]["categories"].head(10).tolist())


=== Cluster 0 ===
['AA,AAA,Electronics Features,Health,Electronics,Health & Household,Camcorder Batteries,Camera & Photo,Batteries,Household Batteries,Accessories,Camera Batteries,Health and Beauty,Household Supplies,Batteries & Chargers,Health, Household & Baby Care,Health Personal Care', 'AA,AAA,Electronics Features,Health,Electronics,Health & Household,Camcorder Batteries,Camera & Photo,Batteries,Household Batteries,Accessories,Camera Batteries,Health and Beauty,Household Supplies,Batteries & Chargers,Health, Household & Baby Care,Health Personal Care', 'AA,AAA,Electronics Features,Health,Electronics,Health & Household,Camcorder Batteries,Camera & Photo,Batteries,Household Batteries,Accessories,Camera Batteries,Health and Beauty,Household Supplies,Batteries & Chargers,Health, Household & Baby Care,Health Personal Care', 'AA,AAA,Electronics Features,Health,Electronics,Health & Household,Camcorder Batteries,Camera & Photo,Batteries,Household Batteries,Accessories,Camera Batteries,Hea

In [134]:
df[df["clusters"] == 0]["categories"].str.split(",").explode().str.strip().value_counts().head(10)

categories
Health                   7456
AA                       3728
Household Batteries      3728
Household & Baby Care    3728
Batteries & Chargers     3728
Household Supplies       3728
Health and Beauty        3728
Camera Batteries         3728
Accessories              3728
Batteries                3728
Name: count, dtype: int64

In [ ]:
tablets_electronics = df[df["clusters"] == 1]["categories"].str.split(",").explode().str.strip().value_counts().head(10)

categories
Fire Tablets                      11196
Tablets                           11196
Amazon Tablets                    11196
Computers & Tablets               11129
All Tablets                       10670
Electronics                        6278
Computers/Tablets & Networking     5126
Tablets & eBook Readers            4986
iPad & Tablets                     3197
Amazon                             3048
Name: count, dtype: int64

In [ ]:
health_beauty_household = df[df["clusters"] == 2]["categories"].str.split(",").explode().str.strip().value_counts().head(10)

categories
Health                   16686
AA                        8343
Robot Check               8343
Household & Baby Care     8343
Batteries & Chargers      8343
Household Supplies        8343
Health and Beauty         8343
Camera Batteries          8343
Accessories               8343
Household Batteries       8343
Name: count, dtype: int64

In [ ]:
tablets = df[df["clusters"] == 3]["categories"].str.split(",").explode().str.strip().value_counts().head(10)


categories
Electronics            8525
Computers & Tablets    4249
Tablets                4035
iPad & Tablets         3910
Music                  3702
Movies                 3702
Tech Toys              3702
Fire Tablets           3200
Amazon Tablets         3200
Kids' Tablets          3101
Name: count, dtype: int64